In [3]:
!pip install striprtf

In [196]:
from google.cloud import storage
from google.cloud import bigquery
from striprtf.striprtf import rtf_to_text

import pandas as pd
import datetime
from pathlib import Path

#### Util objects

In [33]:
storage_client = storage.Client()
bq_client = bigquery.Client()

In [5]:
RESULT_FORMAT = "txt"

In [138]:
def download_rtf(
    bucket_name: str, 
    source_blob_name: str,
    destination_file_name: str,
):
    source_blob_name.download_to_filename(destination_file_name)

In [7]:
def upload_txt_contents(
    bucket_name: str, 
    source_filepath: str, 
    destination_blob_name: str,
):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_filepath)

In [35]:
def convert_rtf_to_text(file_path):
    with open(file_path, 'r') as file:
        rtf_content = file.read()
    text = rtf_to_text(rtf_content)
    return text

In [9]:
def write_converted_file_to_tmp(
    contents: str,
    filepath: str,
):
    with open(filepath, "w", encoding="utf-8") as txt_file:
        txt_file.write(contents)

In [104]:
document_data = pd.read_csv(
    "../court_data/2022/documents.csv", 
    on_bad_lines='skip', 
    engine='python', 
    sep="	",
) #.fillna(method="ffill")
document_data.head(10)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
0,102383845,503.0,5.0,2.0,40467.0,219/14855/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Медінцева Н. М.,NaN,0.0,2022-01-02 00:00:00+02
1,102391042,4856.0,5.0,4.0,40105.0,560/6776/21,2021-12-29 00:00:00+02,2022-01-02 00:00:00+02,Матохнюк Д.Б.,http://od.reyestr.court.gov.ua/files/51/0c9097...,1.0,2022-01-04 00:00:00+02
2,102390645,4824.0,5.0,1.0,NaN,824/245/21,2021-12-22 00:00:00+02,2022-01-02 00:00:00+02,Нежура Вадим Анатолійович,http://od.reyestr.court.gov.ua/files/51/a53df6...,1.0,2022-01-04 00:00:00+02
3,102391822,2670.0,5.0,4.0,40114.0,640/4558/19,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/904bf7...,1.0,2022-01-04 00:00:00+02
4,102391796,2670.0,3.0,4.0,40110.0,640/10043/21,2021-12-07 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/4c9376...,1.0,2022-01-04 00:00:00+02
5,102391789,2670.0,3.0,4.0,40110.0,640/18672/21,2021-12-01 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/f1fc47...,1.0,2022-01-04 00:00:00+02
6,102391769,2670.0,3.0,4.0,40088.0,640/3619/19,2021-11-10 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/98d4ee...,1.0,2022-01-04 00:00:00+02
7,102391764,2670.0,3.0,4.0,40114.0,640/4230/19,2021-11-05 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/dbaa6e...,1.0,2022-01-04 00:00:00+02
8,102391757,2670.0,3.0,4.0,40077.0,640/1726/20,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/145f06...,1.0,2022-01-04 00:00:00+02
9,102388442,1570.0,3.0,4.0,40074.0,420/20761/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Потоцька Н.В.,http://od.reyestr.court.gov.ua/files/51/5514da...,1.0,2022-01-04 00:00:00+02


### Converting from rtf to txt

In [105]:
BUCKET_NAME = "court_data_raw"
RTF_BLOB_FOLDER = "criminal_batch"
TXT_BLOB_FODLER = "criminal_batch_txt"
TMP_FOLDER = "tmp"
RESULT_PREFIX = ""

In [161]:
FULLY_QUALIFIED_NAME = "lab-test-project-1-305710.court_data_2022.document_data"
LOCATION = "us-central1"

In [106]:
criminal_batch = document_data[(document_data["justice_kind"] == 2.0) & (document_data["doc_url"].notna())]
criminal_batch.head(10)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
7591,102391625,1806.0,5.0,2.0,40455.0,592/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Корольова Г. Ю.,http://od.reyestr.court.gov.ua/files/51/21be21...,1.0,2023-01-16 00:00:00+02
7592,102391694,2602.0,5.0,2.0,40455.0,753/25451/21,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Котвицький В. Л.,http://od.reyestr.court.gov.ua/files/51/ac78a2...,1.0,2023-01-16 00:00:00+02
7593,102391699,1511.0,5.0,2.0,40456.0,501/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Семенов О. А.,http://od.reyestr.court.gov.ua/files/51/a56209...,1.0,2023-01-16 00:00:00+02
7594,102391706,814.0,5.0,2.0,40455.0,334/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Фетісов М. В.,http://od.reyestr.court.gov.ua/files/51/4e2053...,1.0,2023-01-16 00:00:00+02
7595,102391745,419.0,5.0,2.0,40455.0,204/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Токар Н. В.,http://od.reyestr.court.gov.ua/files/51/1deff6...,1.0,2023-01-16 00:00:00+02
7596,102385278,2020.0,5.0,2.0,40455.0,641/1/22,2022-01-01 00:00:00+02,2022-01-02 00:00:00+02,Ященко С. О.,http://od.reyestr.court.gov.ua/files/51/8bac1f...,1.0,2023-01-16 00:00:00+02
7597,102391683,2316.0,5.0,2.0,40455.0,712/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Борєйко О. М.,http://od.reyestr.court.gov.ua/files/51/b20ab7...,1.0,2023-01-16 00:00:00+02
7598,102391736,2506.0,5.0,2.0,40455.0,750/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Кузнєцова О. О.,http://od.reyestr.court.gov.ua/files/51/9c7cbc...,1.0,2023-01-16 00:00:00+02
7599,102391695,2606.0,5.0,2.0,40452.0,757/2/22-к,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Смик С. І.,http://od.reyestr.court.gov.ua/files/51/c98b87...,1.0,2023-01-16 00:00:00+02
7600,102391676,1622.0,5.0,2.0,40455.0,554/1/22,2022-01-02 00:00:00+02,2022-01-02 00:00:00+02,Тімошенко Н. В.,http://od.reyestr.court.gov.ua/files/51/b859e2...,1.0,2023-01-16 00:00:00+02


In [164]:
criminal_batch[criminal_batch["doc_id"] == 102383543]

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
7965,102383543,1511.0,5.0,2.0,40455.0,501/5235/21,2021-12-31 00:00:00+02,2022-01-02 00:00:00+02,Тюмін Ю. О.,http://od.reyestr.court.gov.ua/files/51/023273...,1.0,2023-02-13 00:00:00+02


In [188]:
dataset_df = pd.DataFrame(columns=document_data.columns)
dataset_df.head()

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ


In [201]:
def convert_to_parsable_timestamp(tms_source):
    dt = datetime.datetime.fromisoformat(tms_source)
    naive_dt = dt.replace(tzinfo=None)
    formatted_dt = naive_dt.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_dt

In [219]:
def process_document_blob(blob, batch_df, processing_df):
    document_name = blob.name.split("/")[-1]
    print(document_name)
    
    if document_name == "":
        return
    
    document_id = document_name.split(".")[0]
    print("doc id -> ", int(document_id))
    
    new_doc_name = document_id + ".txt"
    processing_data_path = TMP_FOLDER + "/" + document_name 
    
    # 1 - find the document in original dataframe
    doc_row_df = batch_df[batch_df["doc_id"] == int(document_id)]
    doc_row_df["adjudication_date"] = doc_row_df["adjudication_date"].apply(lambda dt: dt.split("+")[0]) 
    doc_row_df["receipt_date"] = doc_row_df["receipt_date"].apply(lambda dt: dt.split("+")[0]) 
    doc_row_df["date_publ"] = doc_row_df["date_publ"].apply(lambda dt: dt.split("+")[0])
    
    if doc_row_df.empty:
        print("data in df is absent -> ", doc_row_df)
        return
    
    # 2 - convert the document from rtf to txt 
    blob.download_to_filename(processing_data_path)
    txt_contents = convert_rtf_to_text(processing_data_path)
    
    parsed_result_filepath = RESULT_PREFIX + document_id + ".txt"
    write_converted_file_to_tmp(txt_contents, TMP_FOLDER + "/" + parsed_result_filepath)  
    
    # 3 - save txt document in blob
    upload_txt_contents(
        BUCKET_NAME,
        TMP_FOLDER + "/" + parsed_result_filepath,
        TXT_BLOB_FODLER + "/" + parsed_result_filepath
    )
    
    # 4 - save the document meta in separate dataframe
    processing_df = pd.concat([processing_df, doc_row_df], ignore_index=True)
    
    # 5 - save the document meta in bigquery
    save_job = bq_client.insert_rows_json(FULLY_QUALIFIED_NAME, [doc_row_df.iloc[0].to_dict()])
    return processing_df

In [216]:
def delete_tmp_folder_contents():
    tmp_folder = Path(TMP_FOLDER)
    for file in tmp_folder.iterdir():
        if file.is_file():
            file.unlink()

In [232]:
def prepare_dataset(bucket_name: str, subfolder: str, original_dataset_df, processing_df):
    """
        1. Takes all files from CloudStorage 
        2. For each blob - 
            a) convert it into the txt format
            b) get the data from dataframe and put the data into bigquery, + another dataframe
        
    """
    # court_data_raw/criminal_batch
    blobs = storage_client.list_blobs(bucket_name, prefix=subfolder)
    
    counter = 0
    for doc_blob in blobs:
        # flush the tmp folder
        if counter % 1000 == 0:
            delete_tmp_folder_contents()
        
        # here should be processing for one document
        processing_df = process_document_blob(doc_blob, original_dataset_df, processing_df)
        print(doc_blob.name)
        
        counter += 1
        if counter > 10:
            return processing_df
        
    return processing_df

In [233]:
dataset_df = prepare_dataset(BUCKET_NAME, RTF_BLOB_FOLDER, criminal_batch, dataset_df)


criminal_batch/
102383543.rtf
doc id ->  102383543


/tmp/ipykernel_397/3450473979.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_row_df["adjudication_date"] = doc_row_df["adjudication_date"].apply(lambda dt: dt.split("+")[0])
/tmp/ipykernel_397/3450473979.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_row_df["receipt_date"] = doc_row_df["receipt_date"].apply(lambda dt: dt.split("+")[0])
/tmp/ipykernel_397/3450473979.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

criminal_batch/102383543.rtf
102383545.rtf
doc id ->  102383545
criminal_batch/102383545.rtf
102383546.rtf
doc id ->  102383546
criminal_batch/102383546.rtf
102383548.rtf
doc id ->  102383548
criminal_batch/102383548.rtf
102383549.rtf
doc id ->  102383549
criminal_batch/102383549.rtf
102383550.rtf
doc id ->  102383550
criminal_batch/102383550.rtf
102383551.rtf
doc id ->  102383551
criminal_batch/102383551.rtf
102383552.rtf
doc id ->  102383552
criminal_batch/102383552.rtf
102383553.rtf
doc id ->  102383553
criminal_batch/102383553.rtf
102383570.rtf
doc id ->  102383570
criminal_batch/102383570.rtf


In [235]:
dataset_df.head(5)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
0,102383543,1511.0,5.0,2.0,40455.0,501/5235/21,2021-12-31 00:00:00,2022-01-02 00:00:00,Тюмін Ю. О.,http://od.reyestr.court.gov.ua/files/51/023273...,1.0,2023-02-13 00:00:00
1,102383545,1512.0,5.0,2.0,40456.0,947/40209/21,2021-12-30 00:00:00,2022-01-02 00:00:00,Чаплицький В. В.,http://od.reyestr.court.gov.ua/files/51/ea34ce...,1.0,2023-02-13 00:00:00
2,102383546,1512.0,5.0,2.0,40456.0,947/40219/21,2021-12-30 00:00:00,2022-01-02 00:00:00,Чаплицький В. В.,http://od.reyestr.court.gov.ua/files/51/f0f3ae...,1.0,2023-02-13 00:00:00
3,102383548,1512.0,5.0,2.0,40455.0,947/40220/21,2021-12-31 00:00:00,2022-01-02 00:00:00,Чаплицький В. В.,http://od.reyestr.court.gov.ua/files/51/f3ceb1...,1.0,2023-02-13 00:00:00
4,102383549,1512.0,5.0,2.0,40456.0,947/4171/21,2021-12-30 00:00:00,2022-01-02 00:00:00,Чаплицький В. В.,http://od.reyestr.court.gov.ua/files/51/68ddff...,1.0,2023-02-13 00:00:00


In [30]:
for index, row in criminal_batch[0:].notna().iterrows():
    if pd.isna(criminal_batch.at[index, "doc_url"]):
        continue
    
    doc_filename = criminal_batch["doc_url"][index].split("/")[-1]
    processing_filename = str(document_data["doc_id"][index]) + ".rtf" # + "_" + doc_filename
    processing_data_path = TMP_FOLDER + "/" + processing_filename 
    
    try: 
        download_rtf(
            BUCKET_NAME,
            RTF_BLOB_FOLDER + "/" + processing_filename,
            processing_data_path,
        )
    except FileNotFoundError as err:
        # print("file not found -> ", processing_filename)
        continue
    
    try:
        txt_contents = convert_rtf_to_text(processing_data_path)
    except:
        # print("Failing parsing: " + processing_data_path)
        continue
    
    parsed_result_filepath = RESULT_PREFIX + processing_data_path + ".txt"
    write_converted_file_to_tmp(txt_contents, parsed_result_filepath)
    
    upload_txt_contents(
        BUCKET_NAME,
        parsed_result_filepath,
        TXT_BLOB_FODLER + "/" + processing_filename + ".txt"
    )

KeyboardInterrupt: 